In [ ]:
from phasespace import GenParticle
import vector
import matplotlib.pyplot as plt

In [ ]:
N_EVENTS = 100_000

BPLUS_M = 5.27965
KAON_MASS = 0.493677
NU_MASS = 0.0


Bplus = GenParticle("B+", BPLUS_M)
Kplus = GenParticle("K+", KAON_MASS)
nu = GenParticle("nu", NU_MASS)
nubar = GenParticle("nubar", NU_MASS)
Bplus.set_children(Kplus, nu, nubar)

In [ ]:
weights, particles = Bplus.generate(N_EVENTS)

In [ ]:
particles

In [ ]:
def get_fourmomenta_slow(momenta):
    """simple implementation with for-loop"""
    momenta = momenta.numpy()
    return [
        vector.obj(px=p[0], py=p[1], pz=p[2], E=p[3])
        for p in momenta
    ]

def get_fourmomenta_fast(momenta):
    """slightly more involved with vector.zip; much faster"""
    momenta = momenta.numpy()
    return vector.zip({
        "px": momenta[:, 0],
        "py": momenta[:, 1],
        "pz": momenta[:, 2],
        "E": momenta[:, 3]
    })

In [ ]:
%%time
p_nu = get_fourmomenta_fast(particles["nu"])
p_nubar = get_fourmomenta_fast(particles["nubar"])

see next example below for how to use the slow / easier variation or how to compute q2 by hand

In [ ]:
q = p_nu + p_nubar

In [ ]:
plt.hist(q.m2, weights=weights)
plt.show()

now do the same but with $B^+\rightarrow\tau^+\nu\rightarrow K^+\nu\bar{\nu}$

In [ ]:
TAU_MASS = 1.776


Bplus = GenParticle("B+", BPLUS_M)
Kplus = GenParticle("K+", KAON_MASS)
nu = GenParticle("nu", NU_MASS)
nubar = GenParticle("nubar", NU_MASS)
tau = GenParticle("tau", TAU_MASS).set_children(Kplus, nubar)
Bplus.set_children(tau, nu)

In [ ]:
weights, particles = Bplus.generate(100000)

let's do the slow variation; getting the four-momenta is easier to code this way (see `get_fourmomenta_slow`) but calculating $q^2$ is a bit more complicated this way

In [ ]:
%%time
p_nu = get_fourmomenta_slow(particles["nu"])
p_nubar = get_fourmomenta_slow(particles["nubar"])

In [ ]:
q2 = [(nu + nubar).m2 for nu, nubar in zip(p_nu, p_nubar)]

In [ ]:
plt.hist(q2, weights=weights)
plt.show()

alternatively, you can also calculate q2 "by hand"

In [ ]:
%%time
q2 = [
    - (nu[0]+nubar[0])**2 - (nu[1]+nubar[1])**2 - (nu[2]+nubar[2])**2 + (nu[3]+nubar[3])**2
    for nu, nubar in zip(particles["nu"].numpy(), particles["nubar"].numpy())
]

In [ ]:
plt.hist(q2, weights=weights)
plt.show()